# Capstone 2

## Polish Bankruptcy

## Pre-processing and Training

## Import Statements

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


## Load Data

In [4]:
bankruptcy_data = pd.read_csv('bankruptcy_data_comb.csv')
data_columns = pd.read_csv('column_key.csv')

dataCorrhigh = pd.read_csv('dataCorrhigh.csv')
dataCorrmid = pd.read_csv('dataCorrmid.csv')
dataCorrlow = pd.read_csv('dataCorrlow.csv')

# save outlier dataframe to file
outlier_df = pd.read_csv('outlier_df.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ski_data.drop(columns='AdultWeekend'), 
                                                    ski_data.AdultWeekend, random_state=5)

In [ ]:
#Call `X_train` and `X_test`'s `fillna()` method, passing `X_defaults_median` as the values to use
#Assign the results to `X_tr` and `X_te`, respectively
X_tr = X_train.fillna(X_defaults_median)
X_te = X_test.fillna(X_defaults_median)

In [ ]:
clf = LogisticRegression()

clf.fit(X_train, y_train) 

ypredict_test = clf.predict(X_test)

ypredict_train = clf.predict(X_train)
